In [1]:
import os
import time
import glob
import pathlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns




In [2]:
os.listdir('/kaggle/input/bbc-news-summary/BBC News Summary/Summaries/') # categories of summaries

['politics', 'sport', 'tech', 'entertainment', 'business']

In [3]:
# Now we define paths for articles, summaries and maintain a list of categories of our data
article_path = '/kaggle/input/bbc-news-summary/BBC News Summary/News Articles'
summary_path = '/kaggle/input/bbc-news-summary/BBC News Summary/Summaries'
categories = ['politics', 'sport', 'tech', 'entertainment', 'business']

In [4]:
def read_in_folders(article_path, summary_path, categories, encoding = "ISO-8859-1"):
    articles = []
    summaries = []
    categs = []
    
    for categs in categories:
        article_paths = glob.glob(os.path.join(article_path, categs, '*.txt'), recursive=True)
        summary_paths = glob.glob(os.path.join(summary_path, categs, '*.txt'), recursive=True)
        print(f'found {len(article_paths)} in articles/{categs} folder, {len(summary_paths)} in summaries/{categs}\n')
        
        if(len(article_paths) != len(summary_paths)):
            print('Both are not equal')
            return 
        
        # put all articles per category in articles
        for index in range(len(article_paths)):
            with open(article_paths[index], mode = 'r', encoding = encoding) as file:
                articles.append(file.read())
                
            with open(summary_paths[index], mode = 'r', encoding = encoding) as file:
                summaries.append(file.read())
                
    return articles, summaries, categs

In [5]:
articles, summaries, categories = read_in_folders(article_path, summary_path, categories)

found 417 in articles/politics folder, 417 in summaries/politics

found 511 in articles/sport folder, 511 in summaries/sport

found 401 in articles/tech folder, 401 in summaries/tech

found 386 in articles/entertainment folder, 386 in summaries/entertainment

found 510 in articles/business folder, 510 in summaries/business

